In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_3D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(x[2], x[1])   
    
    return rt+ndt, (theta1, theta2)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            if a(j*dt) == 0:
                continue
            
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_3D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0] - prms[1]*t
    a2 = lambda t: (a(t))**2
    da2 = lambda t: -2*prms[1] * a(t)
    
    ndt = prms[2]
    mu = np.array([prms[3], prms[4], prms[5]])
    
    T_max = min(max(RT), prms[0]/prms[1])
    fpt = ie_bessel_fpt(a2, da2, mu.shape[0], 0.000001, 
                        dt=0.02, T_max=T_max)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.0001 and rt - ndt < T_max:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])
            term1 = a(rt - ndt) * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'b0_true': [],
               'b0_estimate': [],
               'lambda_true': [],
               'lambda_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': []}

min_b0 = 2
max_b0 = 5

min_lambda = .1
max_lambda = 2

min_ndt = 0.1
max_ndt = 1

min_mu = -3
max_mu = 3

In [6]:
for n in tqdm(range(50)):
    b0 = np.random.uniform(min_b0, max_b0)
    lamb = np.random.uniform(min_lambda, max_lambda)
    a = lambda t: b0 - lamb*t
    
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    RT = []
    Theta = []
    
    for i in range(750):
        rt, theta = simulate_HSDM_3D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_3D_likelihood,
                                     args=(RT, Theta), 
                                     bounds=[(min_b0, max_b0), (min_lambda, max_lambda),
                                             (min_ndt, max_ndt), (min_mu, max_mu),
                                             (min_mu, max_mu), (min_mu, max_mu)])
    
    min_ans = minimize(HSDM_3D_likelihood,
                       args=(RT, Theta),
                       method='nelder-mead',
                       x0=min_ans.x,
                       bounds=[(min_b0, max_b0), (min_lambda, max_lambda),
                               (min_ndt, max_ndt), (min_mu, max_mu),
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    if min_ans.success:
        recovery_df['b0_true'].append(b0)
        recovery_df['lambda_true'].append(lamb)
        recovery_df['ndt_true'].append(ndt)
        recovery_df['mu1_true'].append(mu[0])
        recovery_df['mu2_true'].append(mu[1])
        recovery_df['mu3_true'].append(mu[2])

        recovery_df['b0_estimate'].append(min_ans.x[0])
        recovery_df['lambda_estimate'].append(min_ans.x[1])
        recovery_df['ndt_estimate'].append(min_ans.x[2])
        recovery_df['mu1_estimate'].append(min_ans.x[3])
        recovery_df['mu2_estimate'].append(min_ans.x[4])
        recovery_df['mu3_estimate'].append(min_ans.x[5])

100%|██████████████████████████████| 50/50 [4:05:43<00:00, 294.87s/it]


In [7]:
recovery_df = pd.DataFrame(recovery_df)
recovery_df.corr()

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
b0_true,1.000000,0.919629,0.084408,0.003666,-0.113151,-0.030542,-0.164181,-0.164106,0.299694,0.304156,0.166779,0.167696
b0_estimate,0.919629,1.000000,-0.026760,0.049598,-0.108659,-0.112570,-0.214320,-0.212642,0.274945,0.276539,0.143883,0.143226
lambda_true,0.084408,-0.026760,1.000000,0.910294,0.119015,0.178235,0.064999,0.062929,-0.141931,-0.141329,0.016288,0.017087
lambda_estimate,0.003666,0.049598,0.910294,1.000000,0.121855,0.104481,0.013551,0.013608,-0.154792,-0.157112,0.013815,0.013149
ndt_true,-0.113151,-0.108659,0.119015,0.121855,1.000000,0.973311,-0.087276,-0.087410,-0.232698,-0.227896,0.210031,0.215342
ndt_estimate,-0.030542,-0.112570,0.178235,0.104481,0.973311,1.000000,-0.068080,-0.068995,-0.195909,-0.189865,0.224309,0.230203
mu1_true,-0.164181,-0.214320,0.064999,0.013551,-0.087276,-0.068080,1.000000,0.999440,-0.342110,-0.340805,0.015149,0.018705
mu1_estimate,-0.164106,-0.212642,0.062929,0.013608,-0.087410,-0.068995,0.999440,1.000000,-0.345232,-0.344144,0.017414,0.020951
mu2_true,0.299694,0.274945,-0.141931,-0.154792,-0.232698,-0.195909,-0.342110,-0.345232,1.000000,0.999168,0.156855,0.157668
mu2_estimate,0.304156,0.276539,-0.141329,-0.157112,-0.227896,-0.189865,-0.340805,-0.344144,0.999168,1.000000,0.155242,0.156997


In [8]:
file_name = '_Recovery_data/IE_3d_recovery_linear_750_02.csv'
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
recovery_df

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
0,4.885028,4.895377,0.465765,0.405920,0.662895,0.645148,-1.898659,-1.944745,-0.629421,-0.676649,-0.066508,-0.076919
1,2.892551,3.387810,0.658907,0.992194,0.858844,0.786594,2.761064,2.704808,-0.810218,-0.775703,1.744926,1.740574
2,4.166961,3.688763,0.799790,0.457132,0.452573,0.500997,2.519087,2.482763,1.701095,1.760615,1.239985,1.264259
3,4.118486,4.334070,0.187204,0.235949,0.305405,0.282099,2.490192,2.462265,0.495924,0.486613,-1.780852,-1.824820
4,2.582598,2.379486,1.306909,1.051947,0.857973,0.888572,0.712089,0.740492,1.875774,1.922852,1.307947,1.423371
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2.701276,2.611878,0.462182,0.441379,0.840453,0.880333,1.390451,1.399639,-0.620369,-0.601956,1.654090,1.643247
296,4.644793,3.550613,1.845334,1.221756,0.720188,0.908479,0.719685,0.789882,0.434021,0.489390,2.265123,2.573322
297,4.985022,4.676768,0.667372,0.551459,0.244310,0.310589,1.006810,1.018881,0.671876,0.751351,-2.001806,-2.051728
298,4.686987,4.677691,1.076127,1.040294,0.366462,0.362959,2.696431,2.632685,-0.652036,-0.685561,1.798182,1.815554
